**Table of contents**<a id='toc0_'></a>    
- [パーティクルフィルタによる自己位置推定](#toc1_)    
- [pybulletの起動](#toc2_)    
- [初期設定](#toc3_)    
- [オドメトリクラスの定義](#toc4_)    
- [パーティクルフィルタクラスの定義](#toc5_)    
- [オドメトリに関するパラメータの設定](#toc6_)    
- [センサに関するパラメータの設定](#toc7_)    
- [環境に関する設定](#toc8_)    
- [パーティクルに関するパラメータの設定](#toc9_)    
- [シミュレーション実行](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[パーティクルフィルタによる自己位置推定](#toc0_)

本notebookでは、移動ロボットによるパーティクルフィルタを用いた自己位置推定を行います。


（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import time
import math
import pybullet
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[初期設定](#toc0_)


In [2]:
import random

pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
# time_step = 0.1
pybullet.setTimeStep(time_step)

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[オドメトリクラスの定義](#toc0_)

In [3]:
class WheelOdometry:
    def __init__(self, initial_pose, wheel_radius, wheel_tread, const_right_wheel, const_left_wheel, initial_angle_right_wheel, initial_angle_left_wheel):
        """
        ホイールオドメトリのクラス
        Parameters
        ----------
        initial_pose : tuple
            ロボットの初期位置と姿勢 (x, y, theta)
        wheel_radius : float
            ホイールの半径 [m]
        wheel_tread : float
            左右のホイール間の距離 [m]
        const_right_wheel : float
            右ホイールの定数（摩擦や環境による補正値）
        const_left_wheel : float
            左ホイールの定数（摩擦や環境による補正値）
        initial_angle_right_wheel : float
            右ホイールの初期角度 [rad]
        initial_angle_left_wheel : float
            左ホイールの初期角度 [rad]
        """
        # ホイールのサイズに関する定数
        self.WHEEL_DIAMETER = wheel_radius * 2
        self.WHEEL_TREAD = wheel_tread

        # 環境やタイヤの材質などを考慮した定数
        self.CONSTANT_RIGHT_WHEEL = const_right_wheel
        self.CONSTANT_LEFT_WHEEL = const_left_wheel

        # 1回転あたりの距離
        self.ONE_REVOLUTION_DISTANCE_RIGHT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_LEFT_WHEEL
        self.ONE_REVOLUTION_DISTANCE_LEFT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_RIGHT_WHEEL

        # ロボットの初期位置と姿勢 (x, y, theta)
        self.x = initial_pose[0]
        self.y = initial_pose[1]
        self.theta = initial_pose[2]

        # 前回のホイール角度
        self.last_angle_right_wheel = initial_angle_right_wheel
        self.last_angle_left_wheel = initial_angle_left_wheel

    def update_position(self, current_angle_right_wheel, current_angle_left_wheel):
        """
        ロボットの位置と姿勢を更新する
        
        Parameters
        ----------
        current_angle_right_wheel : float
            右ホイールの現在の角度 [rad]
        current_angle_left_wheel : float
            左ホイールの現在の角度 [rad]
        """
        # 左右のホイールの「回転角度の微小変化量[ΔΘ]」を計算
        delta_angle_right_wheel = current_angle_right_wheel - self.last_angle_right_wheel
        delta_angle_left_wheel = current_angle_left_wheel - self.last_angle_left_wheel

        # 左右のホイールの「回転角度の微小変化量[Δθ]」から、「回転数」を計算（2π → 1回転 となるように変換）
        revolution_right_wheel = delta_angle_right_wheel / (2.0 * math.pi)
        revolution_left_wheel = delta_angle_left_wheel / (2.0 * math.pi)

        # 左右のホイールが「移動した距離[m]」を計算
        distance_right_wheel = revolution_right_wheel * self.ONE_REVOLUTION_DISTANCE_RIGHT
        distance_left_wheel = revolution_left_wheel * self.ONE_REVOLUTION_DISTANCE_LEFT

        # 左右の移動距離から、「平均移動距離[m]」を計算
        distance_avg = (distance_right_wheel + distance_left_wheel) / 2.0

        # ロボットの位置・姿勢を更新
        self.x += distance_avg * math.cos(self.theta)
        self.y += distance_avg * math.sin(self.theta)
        self.theta += math.atan2(distance_right_wheel - distance_left_wheel, self.WHEEL_TREAD)

        # thetaを -π から π の範囲に収める
        if self.theta > math.pi:
            self.theta -= 2 * math.pi
        elif self.theta < -math.pi:
            self.theta += 2 * math.pi

        # 現在のホイール角度を保存
        self.last_angle_right_wheel = current_angle_right_wheel
        self.last_angle_left_wheel = current_angle_left_wheel

    def get_position(self):
        """
        ロボットの位置と姿勢を取得する
        """
        return self.x, self.y, self.theta


# <a id='toc5_'></a>[パーティクルフィルタクラスの定義](#toc0_)
パーティクルフィルタのクラスを定義します。

移動ロボットの自己位置推定におけるパーティクルとは、**実ロボットの分身**のようなもので、ロボットの位置や姿勢を推定するために使用されます。

**分身であるため、実ロボットと同様にオドメトリの計算や、センサによるセンサ値の取得もできるものとして扱います**。  

ただし、パーティクルはあくまで仮想的な分身なので、実ロボットと同じようにオドメトリの計算やセンサ値の取得を行うことはできません。    
そのため、疑似的にオドメトリの計算やセンサ値の取得を行うことで、推定を行います。（詳細は、クラス内の各メソッドの説明を参照してください。）

おおまかな流れは以下の通りです。
1. パーティクルの初期化：各パーティクルをランダムな位置にばらまきます。
    - `__init__(num_particles)`メソッドにて実装
2. パーティクルの位置更新：パーティクルの位置を「運動モデル」に基づいて更新します。（実ロボットのオドメトリに相当） 
   - `particle_filter.update_motion_model(v, omega, time_step)`メソッドにて実装
3. パーティクルのセンサ値の更新：パーティクルに搭載された疑似的なセンサ値を取得します。
    - `particle_filter.measurement_sensor()`メソッドにて実装
4. パーティクルの重みの更新：「パーティクルのセンサ値」と「実ロボットのセンサ値」を比較し、誤差が小さいパーティクルは重みが大きくなるように、重みを更新します。
    - `particle_filter.update_weight(obstacle_distances)`にて実装
5. リサンプリング：重みに基づいてパーティクルを再選定します。これにより、重みの大きいパーティクルの位置にパーティクルが集まる、すなわち、より正確な位置にパーティクルが集まるようになります。
    - `particle_filter.resampling()`メソッドにて実装
7. パーティクルの位置推定：リサンプリングされたパーティクルの位置の重み付き平均をとることで、実ロボットの位置を推定します。
    - `particle_filter.estimate_pose()`メソッドにて実装


In [4]:
import copy

class ParticleFilter():
    class Particle:
        def __init__(self, x: float, y: float, theta: float, num_particles: int):
            """
            パーティクルの初期化
            Parameters
            ---
            x : float
                パーティクルの位置x
            y : float
                パーティクルの位置y
            theta : float
                パーティクルの姿勢θ
            num_particles : int
                パーティクルの数
            """
            # パーティクルの位置と姿勢
            self.x = x
            self.y = y
            self.theta = theta

            # パーティクルと各障害物との距離を格納するリスト
            self.obstacle_distances = []

            # パーティクルの重み
            self.weight = 1. / num_particles

    def __init__(self, init_pos: list, 
                 num_particles: int, particle_range_x: list, particle_range_y: list, particle_range_theta: list,
                 v_noise_dist: float, omega_noise_dist: float, sensor_noise_dist: float, obstacle_positions: list,
                 resampling_x_noise_dist: float, resampling_y_noise_dist: float, resampling_theta_noise_dist: float,
                 draw_particles=False, particle_size=0.02):
        """
        パーティクルフィルタの初期化
        Parameters
        ---
        init_pos : list
            ロボットの初期位置 [x, y, theta]
        num_particles : int
            パーティクルの総数
        particle_range_x : list
            パーティクルの初期位置xの範囲 [min, max]
        particle_range_y : list
            パーティクルの初期位置yの範囲 [min, max]
        particle_range_theta : list
            パーティクルの初期姿勢θの範囲 [min, max]
        v_noise_dist : float
            速度ノイズの分散
        omega_noise_dist : float
            角速度ノイズの分散
        sensor_noise_dist : float
            センサノイズの分散
        resampling_x_noise_dist : float
            リサンプリング時のx方向のノイズの分散
        resampling_y_noise_dist : float
            リサンプリング時のy方向のノイズの分散
        resampling_theta_noise_dist : float
            リサンプリング時のθ方向のノイズの分散
        obstacle_positions : list
            障害物の位置 [[x1, y1], [x2, y2], ...]（障害物は静的であり、動かないものとする）
        draw_particles : bool
            パーティクルの描画を行うかどうか
        particle_size : float
            パーティクルのサイズ（draw_particlesがTrueの場合のみ有効）
        """
        # パーティクルフィルタによる推定位置
        self.x = init_pos[0]
        self.y = init_pos[1]
        self.theta = init_pos[2]

        # パーティクルに関する設定
        self.num_particles = num_particles
        self.v_dist = v_noise_dist
        self.omega_dist = omega_noise_dist
        self.sensor_noise_dist = sensor_noise_dist
        self.resampling_x_noise_dist = resampling_x_noise_dist
        self.resampling_y_noise_dist = resampling_y_noise_dist
        self.resampling_theta_noise_dist = resampling_theta_noise_dist

        # 障害物に関する設定
        self.obstacle_positions = obstacle_positions

        # パーティクルの描画設定
        self.draw_particles = draw_particles

        # パーティクルの初期位置と姿勢をランダムに生成
        self.particles = []
        self.particle_ids = [] # 描画用
        for i in range(self.num_particles):
            x_init = random.uniform(particle_range_x[0], particle_range_x[1])
            y_init = random.uniform(particle_range_y[0], particle_range_y[1])
            theta_init = random.uniform(particle_range_theta[0], particle_range_theta[1])
            particle = self.Particle(x_init, y_init, theta_init, self.num_particles)
            self.particles.append(particle)

            # 描画用
            if self.draw_particles:
                # パーティクルのビジュアルを作成
                particle_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=particle_size, rgbaColor=[0,1,0,1])
                particle_id = pybullet.createMultiBody(0, -1, particle_visual_id, [self.particles[i].x, self.particles[i].y, 0.2], useMaximalCoordinates=True)
                self.particle_ids.append(particle_id)

    def update_motion_model(self, v: float, omega: float, dt: float):
        """
        パーティクルの位置を更新
            実ロボットにおけるオドメトリによる位置更新に相当
            ただし、パーティクルは実ロボットの仮想的な分身であり、エンコーダを用いたオドメトリ情報の取得はできないため、
            実ロボットから得られた「速度」「角速度」をもとに、動作モデルによる位置更新を行う
            また、実ロボットにおけるオドメトリの結果は摩擦などの影響で誤差が生じるため、パーティクルの位置を更新する際にも誤差を加える
        Parameters
        ---
        v : float
            実ロボットの速度
        omega : float
            実ロボットの角速度
        dt : float
            シミュレーションの時間ステップ
        """
        for i in range(self.num_particles):
            # 速度と角速度のノイズを生成
            v_noise = random.gauss(0, self.v_dist)
            omega_noise = random.gauss(0, self.omega_dist)

            # パーティクルの位置を更新
            self.particles[i].x += (v + v_noise) * math.cos(self.particles[i].theta) * dt
            self.particles[i].y += (v + v_noise) * math.sin(self.particles[i].theta) * dt
            self.particles[i].theta += (omega + omega_noise) * dt

            # thetaを -π から π の範囲に収める
            if self.particles[i].theta > math.pi:
                self.particles[i].theta -= 2 * math.pi
            elif self.particles[i].theta < -math.pi:
                self.particles[i].theta += 2 * math.pi

            # 描画用
            if self.draw_particles:
                pybullet.resetBasePositionAndOrientation(self.particle_ids[i], [self.particles[i].x, self.particles[i].y, 0.15], pybullet.getQuaternionFromEuler([0, 0, self.particles[i].theta]))


    def measurement_sensor(self):
        """
        パーティクルにおける障害物との距離を計測（観測値）
            パーティクルのセンサから、センサ値を取得する
            ただし、パーティクルは実ロボットの仮想的な分身であり、実際にセンサを持っているわけではないため、疑似的なセンサにより計測する
            今回は、各障害物の位置を既知としているため、「パーティクルの現在位置」と「各障害物」との距離をセンサ値として扱う
            また、実ロボットのセンサ値はノイズの影響を受けるため、パーティクルの疑似的なセンサ値にもノイズを加える

            （補足）実ロボットがLiDARセンサを使用している場合：
                各パーティクルにも疑似的なLiDARセンサを持たせ、障害物との距離を計測する。
                もちろん、パーティクルはLiDARセンサを実際には持っていないため、疑似的にLiDARを再現する必要がある
                具体的には、環境地図を用意し、その地図情報をもとに距離を計測する
                例えば、地図上でレイキャストを行い、「パーティクルの座標位置」と「黒ピクセル（障害物）の座標位置」までの距離を計測する　等の方法が挙げられる
                （画像上で計測するため、計測結果を実際の距離に変換する必要があることに注意。そのため、距離変換できるような環境地図を用意する必要がある）
        """
        for i in range(self.num_particles):
            # 各障害物との距離を初期化
            self.particles[i].obstacle_distances = []

            # 全ての障害物との距離を計測
            for obstacle_position in self.obstacle_positions:
                # 障害物との距離を計測
                distance = math.sqrt((self.particles[i].x - obstacle_position[0])**2 + (self.particles[i].y - obstacle_position[1])**2)
                distance += random.gauss(0, self.sensor_noise_dist) # センサノイズを加える
                self.particles[i].obstacle_distances.append(distance)
            
    def update_weight(self, obstacle_distance_real_robot: list):
        """
        パーティクルの重みを更新
            各パーティクルにおいて、「実ロボットのセンサ値」と「パーティクルの（疑似的な）センサ値」の誤差を計算し、その誤差をもとに重みを計算する
            より具体的には、両者の誤差が小さいほどそのパーティクルの位置は実ロボットの位置と近そうと予想できるため、重みを大きくする
            逆に、両者の誤差が大きい場合、そのパーティクルの位置は実ロボットの位置とは遠いと予想できるため、重みを小さくする
        Parameters
        ---
        obstacle_distance_real_robot : list
            実ロボットのセンサの計測結果[[1つめのレイの距離], [2つめのレイの距離], ...]
        """
        for i in range(self.num_particles):
            for j in range(len(obstacle_distance_real_robot)):
                error = abs((obstacle_distance_real_robot[j] - self.particles[i].obstacle_distances[j])) # 「実ロボットのセンサにより計測された距離」と「パーティクルのセンサにより計測された距離」の誤差の総和を計算
                self.particles[i].weight *= self.gaussian_probability(error, math.sqrt(self.sensor_noise_dist)) # 誤差からガウス分布に基づく重みを計算（誤差が小さいほど重みが大きくなる）
            self.particles[i].weight += 1e-300  # 重みが0になるのを防ぐための微小な値を加算
        
        # 重みの正規化（resamplingの際に、確率値として扱うために、全てのパーティクルの重みの合計が1になるように正規化）
        sum_weight = sum([particle.weight for particle in self.particles])
        for i in range(self.num_particles):
            self.particles[i].weight /= sum_weight

    def gaussian_probability(self, residual: float, sigma: float):
        """
        ガウス分布に基づく確率密度関数
        Parameters
        ---
        residual : float
            誤差
        sigma : float   
            標準偏差
        
        Returns
        ---
        probability : float
            ガウス分布に基づく確率密度関数
        """
        coefficient = 1.0 / (math.sqrt(2.0 * math.pi) * sigma)
        exponential = math.exp(-0.5 * (residual / sigma) ** 2)
        return coefficient * exponential

    def resampling(self):
        """
        リサンプリング
            重みに基づいてパーティクルをリサンプリングする
            重みが大きいパーティクルは複数回選ばれる確率が高く、逆に重みが小さいパーティクルは選ばれる確率が低くなる
        """
        # リサンプリングに使用する準備
        particle_indices = range(self.num_particles) # パーティクルのインデックス
        tmp_particles = self.particles # パーティクルの一時保存用
        weights = np.array([particle.weight for particle in self.particles]) # 全パーティクルの重み
        rng = np.random.default_rng() # 乱数生成器

        # リサンプリング
        for i in range(self.num_particles):
            # 重みに基づいてパーティクルを選択
            index = rng.choice(particle_indices, p=weights, replace=True) 
            self.particles[i] = copy.deepcopy(tmp_particles[index])

            # ランダムにノイズを加える
            self.particles[i].x += random.gauss(0, self.resampling_x_noise_dist)
            self.particles[i].y += random.gauss(0, self.resampling_y_noise_dist)
            self.particles[i].theta += random.gauss(0, self.resampling_theta_noise_dist)

            # 重みをリセット
            self.particles[i].weight = 1. / self.num_particles
            
            # 描画用
            if self.draw_particles:
                pybullet.resetBasePositionAndOrientation(self.particle_ids[i], [self.particles[i].x, self.particles[i].y, 0.0], pybullet.getQuaternionFromEuler([0, 0, self.particles[i].theta]))

    def estimate_pose(self):
        """
        実ロボットの位置推定
            全パーティクルフィルタの位置、重みの情報をもとに、実ロボットの位置を推定する
        """

        # 全パーティクルの位置と重みを取得
        particle_positions = np.array([[particle.x, particle.y, particle.theta] for particle in self.particles])
        particle_weights = np.array([particle.weight for particle in self.particles])

        # 重み付き平均を計算
        estimated_position = np.average(particle_positions, axis=0, weights=particle_weights) # 重み付き平均を計算
        
        # 推定位置を更新
        self.x = estimated_position[0]
        self.y = estimated_position[1]
        self.theta = estimated_position[2]

    def get_pose(self):
        """
        パーティクルフィルタによる実ロボットの推定位置を返す
        """
        return self.x, self.y, self.theta

# <a id='toc6_'></a>[オドメトリに関するパラメータの設定](#toc0_)

In [5]:
# 左右の車輪のジョイントのインデックス
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# 車輪に関する定数
WHEEL_RADIUS = 0.05 # 車輪の半径(「simple_two_wheel_car.urdf」の車輪の半径と一致させる)
WHEEL_THREAD = 0.325 # 車輪の間隔(「simple_two_wheel_car.urdf」の車輪の間隔と一致させる)
CONSTANT_RIGHT_WHEEL = 1.0 # 右車輪の定数（摩擦や環境による影響を考慮した定数）
CONSTANT_LEFT_WHEEL = 1.0 # 左車輪の定数（摩擦や環境による影響を考慮した定数）

# 並進速度、回転速度のスライダーを作成
linear_vel_slider = pybullet.addUserDebugParameter("linear_velocity", -10, 10, 0)
angular_vel_slider = pybullet.addUserDebugParameter("angular_velocity", -10, 10, 0)

# <a id='toc7_'></a>[センサに関するパラメータの設定](#toc0_)

今回は、直接センサを使用するのではなく、「移動ロボットのセンサリンクの位置」と「各障害物の位置」をシミュレーションから直接取得し、二点間の距離にノイズを加えたものをセンサ値として扱います。

In [6]:
# センサリンクの位置
SENSOR_LINK_IDX = 5
real_sensor_noise_dist = 0.01

# <a id='toc8_'></a>[環境に関する設定](#toc0_)
壁、障害物、移動ロボットを配置します。

（障害物が、移動ロボットに衝突するように生成されてしまった場合、再生成するようにしてください。）

In [7]:
# （四方を囲む）壁のパラメータ
wall_width = 5
wall_height = 0.5
wall_thickness = 0.1

# 障害物のパラメータ
obstacle_num = 10
size_min = 0.1
size_max = 0.3

# 壁の生成
wall_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=[wall_width/2, wall_thickness/2, wall_height/2])
wall_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=[wall_width/2, wall_thickness/2, wall_height/2], rgbaColor=[0.5,0.5,0.5,1])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width/2, -wall_thickness/2, wall_height/2])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width/2, wall_width+wall_thickness/2, wall_height/2])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [-wall_thickness/2, wall_width/2, wall_height/2], pybullet.getQuaternionFromEuler([0, 0, math.pi/2]))
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width+wall_thickness/2, wall_width/2, wall_height/2], pybullet.getQuaternionFromEuler([0, 0, math.pi/2]))

# 障害物の位置をランダムに設定
obstacles = []
for i in range(obstacle_num):
    box_size = random.uniform(size_min, size_max)
    x_init = random.uniform(0+box_size, wall_width-box_size)
    y_init = random.uniform(0+box_size, wall_width-box_size)
    yaw = random.uniform(0, math.pi)
    # 壁で囲まれた領域の中央あたりに障害物がある場合、障害物を配置しなおす
    while (wall_width/2-box_size < x_init < wall_width/2+box_size) and (wall_width/2-box_size < y_init < wall_width/2+box_size):
        x_init = random.uniform(0+box_size, wall_width-box_size)
        y_init = random.uniform(0+box_size, wall_width-box_size)
    obstacles.append([box_size, x_init, y_init, yaw])

# 障害物の配置
obstacle_positions = []
for obstacle in obstacles:
    box_size = obstacle[0]
    x_init = obstacle[1]
    y_init = obstacle[2]
    yaw = obstacle[3]
    obstacle_id = pybullet.loadURDF("../urdf/simple_box.urdf", [x_init, y_init, box_size/2], pybullet.getQuaternionFromEuler([0,0,yaw]), globalScaling=box_size*2, useFixedBase=True)
    pybullet.changeVisualShape(obstacle_id, -1, rgbaColor=[1, 0, 0, 1])
    obstacle_positions.append([x_init, y_init])

# ロボットの読み込み
# 初期位置は四方に囲まれた壁の中央に設定
car_start_x = wall_width/2
car_start_y = wall_width/2
car_start_theta = 0
car_start_position = [car_start_x, car_start_y, 0.1]  # 初期位置(x,y,z)を設定
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, car_start_theta])  # 初期姿勢(roll, pitch, yaw)を設定
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_position, car_start_orientation)

# GUIモードの際のカメラの位置などを設定
camera_distance = wall_width
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [wall_width/2, wall_width/2, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# <a id='toc9_'></a>[パーティクルに関するパラメータの設定](#toc0_)
パーティクルフィルタに関するパラメータを設定します。  

自己位置推定の結果に関わるパラメータであるため、値を変更して結果がどのように変わるか確認してみてください。

In [8]:
# パーティクルの総数
num_particles = 100

# 最初にパーティクルをバラまく範囲を設定
particle_range_x = [car_start_position[0]-0.5, car_start_position[0]+0.5] # ロボットの初期位置から±0.5mの範囲
particle_range_y = [car_start_position[1]-0.5, car_start_position[1]+0.5] # ロボットの初期位置から±0.5mの範囲
particle_range_theta = [car_start_position[2]-math.radians(-10), car_start_position[2]+math.radians(10)] # ロボットの初期姿勢から±10°の範囲

# ノイズの分散を設定
v_noise_dist = 0.01
omega_noise_dist = 0.01
sensor_noise_dist = 0.01

# resamplingのノイズ
resampling_x_noise_dist = 0.01
resampling_y_noise_dist = 0.01
resampling_theta_noise_dist = 0.001

# パーティクルのサイズを設定（描画用）
particle_size = 0.02

# <a id='toc10_'></a>[シミュレーション実行](#toc0_)

シミュレーションを実行すると、パーティクルフィルタによる自己位置推定が行われます。

In [9]:
import sys
# debug用の描画を初期化
pybullet.removeAllUserDebugItems()

# 左右の車輪の回転角度の初期値を取得
initial_angle_right_wheel = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
initial_angle_left_wheel = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

# 左右の車輪の回転角度を初期化
last_odom_x = car_start_x
last_odom_y = car_start_y
last_odom_theta = car_start_theta

# オドメトリクラスのインスタンス化
wheel_odometry = WheelOdometry([car_start_x, car_start_y, car_start_theta], initial_angle_right_wheel, initial_angle_left_wheel, WHEEL_RADIUS, WHEEL_THREAD, CONSTANT_RIGHT_WHEEL, CONSTANT_LEFT_WHEEL)

# パーティクルフィルタクラスのインスタンス化
particle_filter = ParticleFilter([car_start_x, car_start_y, car_start_theta], num_particles, particle_range_x, particle_range_y, particle_range_theta, v_noise_dist, omega_noise_dist, sensor_noise_dist, obstacle_positions, resampling_x_noise_dist, resampling_y_noise_dist, resampling_theta_noise_dist, draw_particles=True, particle_size=particle_size)

# 移動ロボットを初期位置に戻す
pybullet.resetBasePositionAndOrientation(car_id, car_start_position, car_start_orientation)

while True:
    # スライダーの値を取得
    linear_velocity = pybullet.readUserDebugParameter(0)
    angular_velocity = pybullet.readUserDebugParameter(1)

    # 並進速度、回転速度から左右の車輪の指令速度を計算
    right_wheel_velocity = linear_velocity - angular_velocity * WHEEL_THREAD / 2
    left_wheel_velocity = linear_velocity + angular_velocity * WHEEL_THREAD / 2

    # 速度を設定
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # 現在の車輪の回転角度を取得
    current_angle_wheel_right = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
    current_angle_wheel_left = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

    # オドメトリの計算
    wheel_odometry.update_position(current_angle_wheel_right, current_angle_wheel_left)
    odom_x, odom_y, odom_theta = wheel_odometry.get_position()

    # オドメトリから、速度と角速度を計算
    v = math.sqrt((odom_x - last_odom_x)**2 + (odom_y - last_odom_y)**2) / time_step
    omega = (odom_theta - last_odom_theta) / time_step

    # オドメトリの更新
    last_odom_x = odom_x
    last_odom_y = odom_y
    last_odom_theta = odom_theta

    # センサ位置を取得
    sensor_link_position = pybullet.getLinkState(car_id, SENSOR_LINK_IDX)[0]

    # 全ての障害物との距離を計測
    obstacle_distances = []
    for obstacle_position in obstacle_positions:
        # パーティクルと障害物の距離を計算
        #   今回は実際にセンサでは計測せず、「移動ロボットのセンサリンクの位置」と「各障害物の位置」をシミュレーションから直接取得し、二点間の距離にノイズを加えたものをセンサ値として扱う
        #   実際は、「実ロボットのLiDARによる計測結果」と「パーティクルの（疑似的な）LiDARによる計測結果」の誤差を計算し、その誤差をもとに重みを計算することが多い（今回は簡略化のため、センサ値を直接取得）
        distance = math.sqrt((sensor_link_position[0] - obstacle_position[0])**2 + (sensor_link_position[1] - obstacle_position[1])**2) # 「移動ロボットのセンサリンクの位置」と「障害物の位置」の距離
        distance += random.gauss(0, real_sensor_noise_dist) # センサノイズを加える
        obstacle_distances.append(distance)
    
    # パーティクルフィルタの更新（メインとなる処理） ############################################################
    particle_filter.update_motion_model(v, omega, time_step) # 運動モデルにより、各パーティクルの位置を更新（オドメトリによる位置更新に相当）
    particle_filter.measurement_sensor() # 各パーティクルで（疑似的な）センサ値を計測
    particle_filter.update_weight(obstacle_distances) # パーティクルの重みを更新
    particle_filter.resampling() # リサンプリング
    particle_filter.estimate_pose() # パーティクルの位置推定
    #############################################################################################################
    estimated_pose = particle_filter.get_pose() # パーティクルフィルタによる推定位置を取得

    # 推定位置と真の位置を表示
    true_pose = [pybullet.getBasePositionAndOrientation(car_id)[0][0], pybullet.getBasePositionAndOrientation(car_id)[0][1], pybullet.getEulerFromQuaternion(pybullet.getBasePositionAndOrientation(car_id)[1])[2]]
    sys.stdout.write(f"\r【estimated position: ({estimated_pose[0]:.2f}, {estimated_pose[1]:.2f}, {estimated_pose[2]:.2f})】【true position: ({true_pose[0]:.2f}, {true_pose[1]:.2f}, {true_pose[2]:.2f})】               ")

    time.sleep(time_step)

【estimated position: (3.78, 2.78, 0.27)】【true position: (3.81, 2.78, 0.70)】               